<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Send-Data-to-DB" data-toc-modified-id="Send-Data-to-DB-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Send Data to DB</a></span></li><li><span><a href="#Use-of-DB-views" data-toc-modified-id="Use-of-DB-views-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Use of DB views</a></span></li><li><span><a href="#Merged-data-extraction" data-toc-modified-id="Merged-data-extraction-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Merged data extraction</a></span></li></ul></div>

Pipeline for data storage and merging. Intermediate results are displayed in this file but the worker functions are in Preprocess() in preprocessing.py.

In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import settings
import sqlalchemy as sqla
from sqlalchemy.orm import sessionmaker
import time

from sqlalchemy_declarative import *
from preprocessing import Preprocess

PATH = "C:/Users/julia/Documents/coding/codingchallenges/qbiz/Archive/DataHealthProfiles/"
prep = Preprocess()

### Send Data to DB

In [3]:
#populate PostgreSQL database
prep.populate_db(PATH)

48 files gathered...

Sucessfully added acutesexuallytransmittedinfections
Sucessfully added adultssmoking
Sucessfully added alcoholspecifichospitalstaysunder18
Sucessfully added allageallcausemortalityaggregatetrend
Sucessfully added allethnicgroups
Sucessfully added asian
Sucessfully added averagedeprivedquintile
Sucessfully added black
Sucessfully added chinese
Sucessfully added deprivation
Sucessfully added drugmisuse
Sucessfully added earlydeathscanceraggregatetrend
Sucessfully added earlydeathscancer
Sucessfully added earlydeathsheartdiseaseandstroke
Sucessfully added earlydeathsheartdiseasestrokeaggregatetrend
Sucessfully added excesswinterdeaths
Sucessfully added gcseachieved5acincengmaths
Sucessfully added healthyeatingadults
Sucessfully added hipfracturein65sandover
Sucessfully added hospitalstaysforalcoholrelatedharm
Sucessfully added hospitalstaysforselfharm
Sucessfully added incidenceofmalignantmelanoma
Sucessfully added increasingandhigherriskdrinking
Sucessfully added in

### Use of DB views

I am planning to run multivariate models so it will be convenient to keep all indicator data in memory (rather than in the db). This is v. manageable as the data is small ~350x48. The time to extract all indicators from the db is relatively short (~700ms) but this will increase superlinearly with the number of samples. I experimented with views to speed this up but did not attain markedy better performance. See results below FYI. 

It would be overkill here, but with a (much) larger dataset I would use a preprocessing pipeline to create a clean dataset (stored in a db, on disk, or in cloud storage). I would then use PyTorch data loaders to make samples available for training. 

In [6]:
print("Std db access:")
%timeit prep.get_df_from_db_no_view() 
print()
#now create db view
t0 = time.time()
prep.create_db_view() #can only run this once
t1 = time.time()
print("Time taken for view creation:", t1-t0) 
print()

print("db access after creating view:")
%timeit prep.get_df_from_db_w_view()
print()

print("Time taken for std db access after creating view:") 
%timeit prep.get_df_from_db_no_view()  #i.e. using original query

Std db access:
794 ms ± 28.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

View Created
Time taken for view creation: 0.4892246723175049

db access after creating view:
639 ms ± 37.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

Time taken for std db access after creating view:
755 ms ± 28.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


The times for db access reduced slightly by addition of views. 

### Merged data extraction

In [2]:
df = prep.get_df_from_db_w_view()


(361, 47)


                  AREA_NAME ONS_CODE_NEW  obesechildrenyear6      other  \
0                   England    E92000001           19.202513  46.393292   
1                North East    E12000001           22.103695  44.023180   
2                North West    E12000002           19.772661  51.884162   
3  Yorkshire and the Humber    E12000003           19.231373  49.730764   

   peoplediagnosedwithdiabetes  physicallyactiveadults  \
0                     5.758291               56.028024   
1                     6.138118               53.879883   
2                     6.038865               53.492282   
3                     5.903651               55.267176   

   proportionofchildreninpoverty  roadinjuriesanddeaths  smokinginpregnancy  \
0                      21.087867              41.904076           13.307570   
1                      24.793613              35.203446           20.250416   
2                      22.897068              41.947291           17.221996   
3                